# Week 5: Introduction to Supervised Machine Learning

This week, we will introduce another tool that can be used to analyse data: Machine Learning. By the end of this model, you should be able to:
1. Define Supervised Machine Learning and understand its importance in biology.
2. Explain the typical procedure to train an ML model.
3. Implement a program to create an ML Model using a framework called `sklearn`.
4. Understand appropriate ways to evaluate the performance of an ML Model.


### What is Machine Learning?
First, you have probably heard both the terms machine learning (ML) and artificial intelligence (AI) before and may be wondering what the difference is.


**Artificial Intelligence (AI)** is the field of computer science that focuses on creating systems capable of performing tasks that would typically require human intelligence. These tasks include recognizing patterns, learning from data, making decisions, and solving complex problems.

**Machine Learning (ML)** is a subset of AI. Rather than explicitly programming a computer to follow a fixed set of rules, in ML, we "teach" computers to learn from data. This is particularly useful in fields like biology, where there is an abundance of data, but the patterns and relations in the data are complex and hard for a human to parse through.


![aiml_hierarchy](aiml_hierarchy.png)

AI and ML are becoming essential tools in biology, with applications such as:

* Genomics: Identifying gene sequences associated with specific diseases.
* Drug Discovery: Predicting which compounds may act effectively on certain targets, helping to speed up the drug discovery process.
* Imaging and Diagnostics: Analyzing medical images (like MRI or histology slides) to identify patterns that might indicate disease.
* Ecology and Evolution: Analyzing large-scale environmental data or understanding population dynamics and evolutionary patterns.


Many of you have heard recent developments in ML, and these advancements continue to impact the way we work, learn, and coexist with technology. You may be surprised to find that machine learning is becoming increasingly important in biological research. A study by  [Walsh et al. (2021)](https://https://www.nature.com/articles/s41592-021-01205-4#citeas) shows an exponential increase of ML publications in biology since the 1990's, shown in the figure below.

![ml_in_bio](ml_in_bio.png)

It's clear to see the scientific community is rapidly adopting ML techniques as a means to drive new findings.  Bridging the gap in knowledge between computer scientists/engineers and researchers in the scientific community is very much a work in progress. Our goal is to learn, at a very high level, what machine learning entails and how it may be used in biological research.


### ML Pipeline

A Machine learning pipeline typically involves 4 steps.

1. Data Preparation: In this step, we obtain the relevant data for the task we are trying to perform and clean up records that seem incorrect.
2. Data Exploration: We then analyze the data at hand to manually find potentially interesting patterns.
3. Model Training: Once we have explored our data and manually identified potential trends and patterns, we can train (aka fit) a machine learning model. Ideally, the ML model will pick up patterns we have missed and will be able to outperform rules we discover.
4. Model Evaluation: To confirm if the model picked up useful trends, we will use a variety of metrics to evaluate how well the model does at our task.

In this course, Week 3 covered Steps 1 and 2 (but we will practice step 2 below). We will cover Steps 3 and 4 in the main module.

![mlp](ml_pipeline.png)

### Problem statement: Breast cancer

Hopefully, you enjoyed the heart failure analysis from earlier weeks, because we have yet another task:

You have recently moved to a new research lab within the Canadian Cancer Society, studying breast cancer cells. The Canadian Cancer Society estimates that 1 in 8 women will develop breast cancer, with breast cancer constituting about 14% of cancer deaths among women. However, not all breast cancer cells are malignant. If there is a way to accurately detect if a particular breast cancer cell is benign or malignant early on, medical resources could be better allocated for better patient care. The team would like to find out if machine learning could be used to predict which cancer cells are benign or malignant, knowing that ML methods have been  shown to be feasible in other cancer cell analyses in the industry.

You have been given a public dataset from the Diagnostic Wisconsin Breast Cancer Database*, containing features from digitized images of breast cancer cell nuclei. From these images, the research team took measurements that show various characteristics (eg. radius, symmetry, etc.) for each sample. For each characteristic, they also took several measurements on each image and computed the mean, standard error, and "worst" (mean of the three largest values). These measurements and mean/std error/worst values comprise the features of this dataset. For example, the ```radius_mean``` feature is the mean of distances from the center of the nucleus to various points on the perimeter. ```radius_worst``` is the mean of the 3 longest distances from the nucleus to the center for that image. 

Take a moment to view the dataset **bc_data.csv** (using Excel or another .csv file viewer). 

While you are excited to contribute to your team's research, there are a few gaps in your knowledge, starting with... how does machine learning even work?

*https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data

### Data Preparation
![dp](data_prep_pipeline.png)
While in most real-world cases, you (or your team) may need to go out and collect data, we will use a dataset that has already been prepared for us. Below, we load in the dataset from an online source where `X` is the measurements taken by the researchers, and `y` contains whether or not a positive diagnosis was given. Each row in the data represents one sample.

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('bc_data.csv', index_col=0)
df = df.drop('Unnamed: 32', axis=1)
df.head()

y = df["diagnosis"] #.map({"B" : 0, "M" : 1})
X = df.drop("diagnosis", axis = 1)


Below we print out the first few rows of both `X` (the measurements taken by the researchers) and `y` (the diagnosis for diabetes)

In [ ]:
X.head()

In [ ]:
y.head()

### Data Exploration:

![de](data_explore_pipeline.png)

Before we dive into training an ML Model, we will first manually explore the data to see if we can identify potential reasons for diabetes.

Let's analyze the positive and negative cases for the Breast Cancer dataset. Under the column "diagnosis", there are 2 possible values: M (malignant) and B (benign). 

---
**Q*1: Subset `X` into two different dataframes, one with all positive cases and one with negative cases. What percentage of patients have diabetes?**

> Hint: Look at week 3 Question 5 on how to subset dataframes.

<span style="background-color: #FFD700">**Write your code below**</span> 


In [ ]:
positive_cases = ...
negative_cases = ...
n_positive = ...
n_negative = ...

print(f"{n_positive / (n_positive + n_negative) * 100:.2f}% of the cases are positive")

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**Q*2: Calculate the mean and standard deviations of the attributes for patients with diabetes as well as the average values for patients without diabetes. What differences do you notice between the two groups?**

<span style="background-color: #FFD700">**Write your code below**</span> 


<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

**Q*3: Create Histograms of the patient attributes for both the positive and negative groups. What differences do you notice between the two groups?**

<span style="background-color: #FFD700">**Write your code below**</span> 


In [ ]:
...
plt.show()

In [ ]:
...
plt.show()

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---

## **Graded Question: (3 marks)**

**GQ*1. (3 marks) Using the analysis above, do your best to create a few rules (1-2+) that doctors may use to identify patients with breast cancer, then subset `X` and `y` based on these rules (1pt). How many patients with breast cancer did you identify (1pt)? How many did you miss (1pt)?**

> Rule Examples: `Patients have diabetes if Age > 10` or `Patients don't have diabetes if Pregnancies >=2`. These do not do well, please make your own.


In [ ]:
# Example rule
example_rule_positive_cases = y[X['area_worst'] > 600]
example_rule_negative_cases = y[X['area_worst'] <= 600]

In [ ]:
n_patients_captured = np.sum(example_rule_positive_cases == 'M') 
n_patients_missed = np.sum(example_rule_negative_cases == 'B') 
print(f"Ratio of Patients captured: {n_patients_captured:.2f}, Patients missed: {n_patients_missed:.2f}")

<span style="background-color: #FFD700">**Write your code below**</span> 


In [ ]:
...

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer here:

---